In [1]:
import pandas as pd
from pandasql import sqldf
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import cross_val_score
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
pd.set_option('display.max_colwidth', 70)
pd.set_option('display.max_rows', None)

In [3]:
pysqldf = lambda q: sqldf(q, globals())

In [4]:
#nltk.download('stopwords')
#nltk.download('wordnet')

In [5]:
# setting a couple of workbook variables
text_source = "transcript"
tag = "opioid"

In [6]:
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [7]:
df_human = pd.read_csv("data/classifications.csv")
df_ml = pd.read_csv("data/cleaned_data.csv")

In [8]:
df_ml.head()

,ID,title,runtime,transcript
0,ffxh0257,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17,record okay took lunch break mrs dilber talking exhibit number pag...
1,fghb0039,Camel Cigarettes informercial,00:20:51,getting pretty late hit trail snoop around living like ain t job k...
2,fgxh0257,"Deposition of Mark Trudeau, President and CEO",0:52:40,back record questions witness concludes mr trudeau composition tha...
3,fhfk0146,CNN Crossfire debate on smoking in public,0:26:20,tonight washington crossfire saturday smoking skies ifs ands butts...
4,fhgb0191,Think. Don't Smoke. ad campaign compilation,0:34:02,hey guys may ask couple questions ever tried cigarettes know reall...


In [9]:
df_human.head()

,ID,description,subject,title,runtime
0,ffhb0039,Dan Chenowetch is interviewed about why he smokes and where he has...,tobacco; cigarette; secondhand smoke; addiction; bans; flavors; br...,Interview with smoker Dan Chenowhich,0:20:41
1,ffxh0257,Deposition of Jeff Kilper in RE: National Prescription Opiate Liti...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17
2,fghb0039,Camel Cigarettes infomercial on why their cigarettes are superior ...,tobacco; cigarette; marketing;,Camel Cigarettes informercial,00:20:51
3,fgxh0257,Deposition of Mark Trudeau in Re: State of Rhode Island v Purdue P...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Mark Trudeau, President and CEO",0:52:40
4,fhfk0146,CNN debate on the ban of smoking on Airplanes. Ban is put in place...,tobacco; cigarette;secondhand smoke; closed environment;,CNN Crossfire debate on smoking in public,0:26:20


In [10]:
# Todo? do we want to denormalize this, or get a list of distinct subjects

In [11]:
df_dataset = pysqldf(f"""
SELECT dh.*, dml.{text_source}
FROM 
    df_human dh
JOIN
    df_ml dml
ON dh.ID = dml.id
    WHERE description is not NULL
    AND description != 'error code 224003'
    AND subject NOT LIKE '%href%'
""")

In [12]:
len(df_dataset)

324

In [13]:
df_dataset.head(2)

,ID,description,subject,title,runtime,transcript
0,ffxh0257,Deposition of Jeff Kilper in RE: National Prescription Opiate Liti...,lawsuit; legal activity; litigation; opioids; sales; marketing;,"Deposition of Jeff Kilper, Senior Director for Finance for Special...",1:07:17,record okay took lunch break mrs dilber talking exhibit number pag...
1,fghb0039,Camel Cigarettes infomercial on why their cigarettes are superior ...,tobacco; cigarette; marketing;,Camel Cigarettes informercial,00:20:51,getting pretty late hit trail snoop around living like ain t job k...


In [14]:
#df_dataset.set_index("ID", inplace=True)

In [15]:
# what is this doing and can i use python instead? i can't figure out how to
# selecting ID, subject, transcript
# adding category column when subject includes wanted tag, then 1, else 0

# df = pysqldf(f"""
# SELECT 
#     ID,
#     subject,  
#     {text_source},
#     CASE
#         WHEN subject LIKE '%lawsuit%' THEN 1
#         ELSE 0
#     END AS category
# FROM df_dataset
# --WHERE subject LIKE '%lawsuit%'
# """)

df = df_dataset[['ID', 'subject', 'transcript']]
category = []
for row in df['subject']:
    if tag in row:
        category.append(1)
    else:
        category.append(0)
df['category'] = category
df

/var/folders/bs/k89rn4w909bgbnyc21cw85h80000gn/T/ipykernel_69376/1314360593.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category'] = category


,ID,subject,transcript,category
0,ffxh0257,lawsuit; legal activity; litigation; opioids; sales; marketing;,record okay took lunch break mrs dilber talking exhibit number pag...,1
1,fghb0039,tobacco; cigarette; marketing;,getting pretty late hit trail snoop around living like ain t job k...,0
2,fgxh0257,lawsuit; legal activity; litigation; opioids; sales; marketing;,back record questions witness concludes mr trudeau composition tha...,1
3,fhfk0146,tobacco; cigarette;secondhand smoke; closed environment;,tonight washington crossfire saturday smoking skies ifs ands butts...,0
4,fhgb0191,tobacco; cigarette; youth access; PSA; children; future;,hey guys may ask couple questions ever tried cigarettes know reall...,0
5,fhhb0039,tobacco; cigarette; advertise; appeal;,hello doug sarah mcclurkin think time program talked know one ciga...,0
6,fhhh0100,tobacco; cigarette; break; relief;,ever heard man literally walk wall human fly well looking hundred ...,0
7,fhhj0223,tobacco; cigarette; youth; regulation;,top story tonight indiana retail associations joining forces keep ...,0
8,fhwm0256,opioids; coupon program; efficacy; sales and marketing;,tell remember detail happened iraq benefits drug used okay force r...,1
9,fjhb0039,tobacco; cigarette;quality; advertising;,welcome winston bull comedy break guys gonna right watching gerald...,0


In [16]:
df.set_index("ID", inplace=True)

In [45]:
df.head()

28

In [47]:
# number of videos that have chosen tag
(df.category == 1).sum()

28

In [18]:
df_train, df_test = train_test_split(df, test_size=0.5)

In [19]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words=stop)),
    ('clf', RandomForestClassifier())
    ])

In [20]:
X_train = text_clf['vect'].fit_transform(df_train[text_source])

In [21]:
text_clf['vect'].get_feature_names_out()

array(['aaron', 'abandon', 'abandoned', ..., 'zooming', 'zr', 'zucker'],
      dtype=object)

In [22]:
#X_train.todense()

In [23]:
df_vector = pd.DataFrame(X_train.todense())
df_vector.columns = text_clf['vect'].get_feature_names_out()

In [24]:
pd.set_option('display.max_rows', None)
#df_vector.T

In [25]:
#df_train.iloc[0][text_source]

In [26]:
X_train = text_clf.fit(df_train[text_source], df_train['category'])

In [27]:
feature_df = pd.DataFrame({"feature": text_clf['vect'].get_feature_names_out(), 
                           'importance': text_clf['clf'].feature_importances_})

In [28]:
pysqldf("""
SELECT 
    *
FROM
    feature_df
ORDER BY
    importance DESC
""").head(10)

,feature,importance
0,email,0.022419
1,mallinckrodt,0.016505
2,familiar,0.012267
3,page,0.011239
4,generics,0.010906
5,included,0.010865
6,oxycodone,0.010488
7,discussed,0.009625
8,physicians,0.009355
9,provided,0.009315


In [29]:
feature_df.sort_values(by=['importance'], ascending=False)

,feature,importance
4111,email,2.241868e-02
7512,mallinckrodt,1.650488e-02
4662,familiar,1.226674e-02
8805,page,1.123918e-02
5312,generics,1.090557e-02
6258,included,1.086475e-02
8778,oxycodone,1.048783e-02
3632,discussed,9.624867e-03
9156,physicians,9.354835e-03
9763,provided,9.315421e-03


In [30]:
cross_val = cross_val_score(text_clf, df_test[text_source], df_test['category'], cv=4)

In [31]:
cross_val

array([0.92682927, 0.95121951, 0.925     , 0.925     ])

In [32]:
y_proba = text_clf.predict_proba(df_test[text_source])
y_pred = text_clf.predict(df_test[text_source])

In [33]:
y_proba

array([[0.98, 0.02],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.93, 0.07],
       [0.97, 0.03],
       [0.89, 0.11],
       [0.95, 0.05],
       [0.92, 0.08],
       [1.  , 0.  ],
       [0.98, 0.02],
       [0.9 , 0.1 ],
       [0.96, 0.04],
       [0.98, 0.02],
       [1.  , 0.  ],
       [0.98, 0.02],
       [0.98, 0.02],
       [0.98, 0.02],
       [0.95, 0.05],
       [0.96, 0.04],
       [0.99, 0.01],
       [0.99, 0.01],
       [0.93, 0.07],
       [0.6 , 0.4 ],
       [0.92, 0.08],
       [0.98, 0.02],
       [1.  , 0.  ],
       [0.82, 0.18],
       [0.98, 0.02],
       [0.95, 0.05],
       [0.97, 0.03],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.9 , 0.1 ],
       [1.  , 0.  ],
       [0.95, 0.05],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.92, 0.08],
       [0.95, 0.05],
       [0.88, 0.12],
       [0.89, 0.11],
       [0.83, 0.17],
       [0.95, 0.05],
       [0.96, 0.04],
       [0.96, 0.04],
       [0.49, 0.51],
       [0.95, 0.05],
       [0.98,

In [34]:
text_clf.classes_

array([0, 1])

In [35]:
prob_no_match = [p[0] for p in y_proba]
prob_match = [p[1] for p in y_proba]

In [36]:
df_test['prob_no_match'] = prob_no_match
df_test['prob_match'] = prob_match
df_test['y_pred'] = y_pred

In [37]:
# rename y_match?
y_adj = []

for p in prob_match:
    if p <= .5:
        y_adj.append("Not")
    else:
        y_adj.append("Match")
    
df_test['y_adj'] = y_adj

In [38]:
df_test[['category', 'y_pred', 'y_adj', 'prob_no_match', 'prob_match']]

,category,y_pred,y_adj,prob_no_match,prob_match
ID,,,,,
yfwk0146,0,0,Not,0.98,0.02
xzgp0190,0,0,Not,1.00,0.00
jgcw0111,0,0,Not,1.00,0.00
fjhb0039,0,0,Not,0.93,0.07
ypgp0190,0,0,Not,0.97,0.03
xgwv0183,0,0,Not,0.89,0.11
ylwk0146,0,0,Not,0.95,0.05
fhfk0146,0,0,Not,0.92,0.08
xrgp0190,0,0,Not,1.00,0.00


In [39]:
pysqldf("""SELECT * FROM df_test WHERE `y_pred` != category""")

,ID,subject,transcript,category,prob_no_match,prob_match,y_pred,y_adj
0,zgwm0256,opioids; physician interview; sales and marketing,okay quick look saw yesterday well moa think see i m assuming neph...,1,0.90,0.10,0,Not
1,fzhh0257,lawsuit; legal activity; litigation; opioids; sales; marketing;,back video record beginning media number five time currently p m o...,1,0.60,0.40,0,Not
2,yfxh0257,lawsuit; legal activity; litigation; opioids; sales; marketing,back record p m mr ross recall testimony earlier today regarding m...,1,0.83,0.17,0,Not
3,ghwm0256,opioids; interview; physician; sales and marketing,let s address cost issue couple moments talk yesterday want ask co...,1,0.90,0.10,0,Not
4,tthh0257,lawsuit; legal activity; litigation; opioids; sales; marketing,time p m clean record mr morelli testified i ll read statement ans...,1,0.84,0.16,0,Not
5,zthh0257,lawsuit; legal activity; litigation; opioids; sales; marketing,back record okay i m going mark next exhibit car daddy exhibit exh...,1,0.67,0.33,0,Not
6,hhwm0256,opioids; physician interview; sales and marketing; constipation,patient included energy tell one one big side effects narcotics co...,1,0.97,0.03,0,Not
7,xhwm0256,opioids; sales and marketing; physician interview; costs,make comment rep saying kind addressed upfront gave little preview...,1,0.95,0.05,0,Not
8,kfwm0256,opioids; sales and marketing; focus group,reported numbers program take somebody important want work unders...,1,1.00,0.00,0,Not
9,fgxh0257,lawsuit; legal activity; litigation; opioids; sales; marketing;,back record questions witness concludes mr trudeau composition tha...,1,0.89,0.11,0,Not


In [40]:
pysqldf(f"""
SELECT df_test.*, df.{text_source}, df.category
FROM df_test
JOIN df
ON df_test.ID = df.ID
""")

,ID,subject,transcript,category,prob_no_match,prob_match,y_pred,y_adj,transcript,category
0,yfwk0146,tobacco; cigarette; advertising; controversy; debaete; politics,welcome back market wrap hour new york i m doug ramsey business fo...,0,0.98,0.02,0,Not,welcome back market wrap hour new york i m doug ramsey business fo...,0
1,xzgp0190,tobacco; cigarette; news,i m hugh downs,0,1.00,0.00,0,Not,i m hugh downs,0
2,jgcw0111,tobacco; cigarette;,september th city portland oregon waited anxiously sundown married...,0,1.00,0.00,0,Not,september th city portland oregon waited anxiously sundown married...,0
3,fjhb0039,tobacco; cigarette;quality; advertising;,welcome winston bull comedy break guys gonna right watching gerald...,0,0.93,0.07,0,Not,welcome winston bull comedy break guys gonna right watching gerald...,0
4,ypgp0190,tobacco; cigarette,word alternate sponsor parliament cigarettes man knows value extra...,0,0.97,0.03,0,Not,word alternate sponsor parliament cigarettes man knows value extra...,0
5,xgwv0183,tobacco; cigarette,minnesota launched backup brownlow buffalo created need going draf...,0,0.89,0.11,0,Not,minnesota launched backup brownlow buffalo created need going draf...,0
6,ylwk0146,tobacco; cigarette; news; reporting,moments rich taliban remember wonderful used make seem charge toba...,0,0.95,0.05,0,Not,moments rich taliban remember wonderful used make seem charge toba...,0
7,fhfk0146,tobacco; cigarette;secondhand smoke; closed environment;,tonight washington crossfire saturday smoking skies ifs ands butts...,0,0.92,0.08,0,Not,tonight washington crossfire saturday smoking skies ifs ands butts...,0
8,xrgp0190,alcohol; commerical; advertisement; promotion,vinyl horse year s hits team thunder thunder maybe next year welco...,0,1.00,0.00,0,Not,vinyl horse year s hits team thunder thunder maybe next year welco...,0
9,fmpj0015,tobacco; cigarette; appeal; advertise; youth;,okay important people disco people back company gets start running...,0,0.98,0.02,0,Not,okay important people disco people back company gets start running...,0


In [41]:
y_pred = text_clf.predict(df_test[text_source])

In [42]:
# everything below this line will vary based on the run

In [43]:
precision, recall, fscore, train_support = precision_recall_fscore_support(df_test['category'], y_pred, pos_label=1, average='binary')
print('Precision: {} / Recall: {} / Accuracy: {} / fscore: {}'.format(
    round(precision, 3), round(recall, 3), (round((y_pred==df_test['category']).sum()/len(y_adj), 3)), round(fscore, 3)))

Precision: 1.0 / Recall: 0.167 / Accuracy: 0.938 / fscore: 0.286


In [44]:
print('f-score', 2 * ((precision * recall) / (precision + recall)))

f-score 0.2857142857142857
